 This is a PoC using the Pydantic models generated from `datamodel-codegen` to recreate the `qhub-linter.yaml` GitHub Action. It was initially created from [Pydantic models that were hand-coded](https://github.com/Quansight/qhub/blob/4f2802524ad28ea07597f7ad8dd18abe83973020/qhub/provider/cicd/github.py#L131).

In [ ]:
import requests 

def download_schema(url, filename):
    with open(filename, "wb") as f:
        content = requests.get(url).content
        f.write(content)

In [ ]:
url = "https://json.schemastore.org/github-workflow.json"
filename = "gha-json-schema.json"

download_schema(url, filename)

In [ ]:
!pip install datamodel-code-generator

In [ ]:
!datamodel-codegen --input gha-json-schema.json --output models.py --allow-population-by-field-name

In [ ]:
import models

In [ ]:
def get_json(model):
    return model.json(
        indent=2,
        by_alias=True,
        exclude_unset=True,
        exclude_defaults=True,
    )

In [ ]:
def pip_install_qhub(version):
    return f"pip install qhub=={version}"

In [ ]:
qhub_version = "0.4.3"
PYTHON_VERSION = "3.10"

In [ ]:
step1 = models.Step(
    name="Checkout Image",
    uses="actions/checkout@master",
    with_={"token": "${{ secrets.REPOSITORY_ACCESS_TOKEN }}"},
)

In [ ]:
step2 = models.Step(
    name="Set up Python",
    uses="actions/setup-python@v2",
    with_={"python-version": PYTHON_VERSION},
)

In [ ]:
step3 = models.Step(
    name="Install QHub", 
    run=pip_install_qhub(qhub_version)
)

In [ ]:
step4_envs = {
    "PR_NUMBER": "${{ github.event.number }}",
    "REPO_NAME": "${{ github.repository }}",
    "GITHUB_TOKEN": "${{ secrets.REPOSITORY_ACCESS_TOKEN }}",
}

step4 = models.Step(
    name="QHub Lintify",
    run="qhub validate --config qhub-config.yaml --enable-commenting",
    env=step4_envs,
)

In [ ]:
jobs = models.NormalJob(
    name="qhub",
    runs_on="ubuntu-latest",
    steps=[step1, step2, step3, step4],
)

In [ ]:
refs = models.Ref1(
    branches=["main"],
    paths=["qhub-config.yaml"]
)

In [ ]:
on = models.OnItem(
    pull_request=refs
)

In [ ]:
qhub_linter = models.Model(
    name="qhub linter",
    on=on,
    jobs={"qhub-validate": jobs},
)

In [ ]:
print(get_json(qhub_linter))

In [ ]:
fn = "qhub-linter-new.yaml"

with open(fn, "w") as f:
    f.write(get_json(qhub_linter))